<a href="https://colab.research.google.com/github/DiDevv/Vector-Database/blob/main/Similarity_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução:

Todo esse código estará armazenado no meu Github com o intuito tanto de guardar o meu aprendizado, quanto de compartilhar para outras pessoas.

Todo aprendizado aquiduirido aqui me foi proporcionado gratuitamente pela organização DeeplearningAI. O curso pode ser encontrado aqui: [Building Applications With Vector Databases](https://learn.deeplearning.ai/courses/building-applications-vector-databases)

A explicação do código estará em português e algumas abordagens alternativas foram tomadas pois estou rodando os códigos em um ambiente meu e não dentro do jupyter do próprio curso.

O curso ainda não possui legendas em português, foi um ponto de dificuldade para mim pois ainda estou aprimorando minha escuta do idioma inglês, por isso, tentarei abordar o conteúdo em português por aqui, para quebrar a barreira linguística para outras pessoas que ainda não conseguem "se virar" no inglês :P, eu te entendo!

Espero que possa ser útil, tanto pra você quanto pra mim!

# Busca por Similaridade - Módulo 1

Esse módulo trata do nosso primeiro contato com o Pinecone e com um dos casos de uso mais frequentes quando se utiliza um banco de dados vetorial: Busca por Similaridade.

Como o próprio nome já diz, a busca por similaridade é o processo de buscar conteúdos similares no banco de dados. Pra isso, precisamos primeiro "embeddar" nosso conteúdo - transformação de palavras, frases ou itens em vetores em um espaço vetorial (as aulas de algebra linear nunca fizeram tanto sentido) - e depois enviarmos nossos embeddings para o banco de dados vetorial, assim, podendo escolher um método de busca por similaridade (nesse curso utilizamos o cosseno da similaridade)!

---

Antes de prosseguir, é necessário instalar alguns pacotes!

In [71]:
%pip install datasets sentence_transformers pinecone dlai-tools tqdm

# Importação das Bibliotecas


In [72]:
import warnings
warnings.filterwarnings('ignore')

In [73]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer # Modelos baseados no Transformer e possui uma forma mais fácil de computar vetores que representam sentenças, paragrafos e imagens
from pinecone import Pinecone, ServerlessSpec

import os
import time
import torch
import getpass

In [74]:
from tqdm.auto import tqdm # Para acompanhar o progresso do código

# Vamos codar!

In [75]:
dataset = load_dataset("quora", split="train[240000:290000]") # Baixando o conjunto de dados Quora, e selecionando os dados específicos do split train que vão do 240000 ao 290000

In [76]:
dataset[:3] # Visualizando alguns itens do dataset

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]},
  {'id': [33183, 351730],
   'text': ['Which is the best smartphone under 20K in India?',
    'Which is the best smartphone with in 20k in India?']},
  {'id': [351731, 351732],
   'text': ['Steps taken by Canadian government to improve literacy rate?',
    'Can I send homemade herbal hair oil from India to US via postal or private courier services?']}],
 'is_duplicate': [False, True, False]}

---

Vamos tratar esse dataset e separar apenas as questions dentro de uma lista.

In [77]:
questions = []
for record in dataset['questions']:
    questions.extend(record['text'])
questions = list(set(questions)) # O objetivo em converter primeiro em Set e depois em lista é remover duplicatas, pois sets não podem possuir conteúdos duplicados
print("\n".join(questions[:10])) # O join serve pra concatenar uma lista com um separador específico, nesse caso, uma quebra de linhas :)
print('-' * 50)
print(f"nunmber of questinons {len(questions)}")

Will Donald Trump's brand value decrease if he loses in the presidential race?
What is the salary for a software engineering intern with MS (Computer Science), Computer Science PhD and Operations Research PhD in Microsoft (US)?
What are some of the best career options in India?
How do I get better at acting?
What strikes first time visitors as special or unusual when they arrive in Chula Vista, CA?
How can I alleviate hamstring pain while stretching after lifting?
How can I recover my Facebook email account if someone change my password without losing my Facebook account?
What kind of nlp research is happening at Facebook?
How can be a good leader?
What’s the best breakfast in the morning?
--------------------------------------------------
nunmber of questinons 88919


## Vamos criar e testa nosso Embedding!

Essa célula verifica se podemos usar CUDA (pra quem tem placa de vídeo ^^), para processar os dados mais rapidamente.
Se você não possuir,a tranquilo, o conjunto de dados é pequenininho! :)





In [78]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print("Desculpa, sem cuda por aqui :P")
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Embeddando uma query de exemplo. Podemos notar que aqui teremos 384 elementos nesses embeddings, e se imprimirmos a variável, conseguiremos ver as coordenadas de todos os embeddings no espaço vetorial!

In [79]:
query = 'which city is the most populated in the world?'
embedding_test = model.encode(query)
print(embedding_test)
print("--"*50)
print(embedding_test.shape)

[ 1.50860831e-01  1.29710427e-02 -5.08222543e-02  6.06876425e-02
 -3.35402234e-04 -2.90263258e-02  5.06248511e-02  2.99537871e-02
 -3.10023613e-02  5.96766472e-02  5.95133491e-02 -1.33624226e-01
  7.15945708e-03  4.92674410e-02  1.59945562e-02 -2.03522649e-02
  3.74022759e-02 -9.18012261e-02  7.44751915e-02 -5.41728362e-02
 -5.14636040e-02 -4.53633517e-02  6.31610677e-02  4.29923683e-02
  2.43182424e-02  2.65026204e-02  1.74598321e-02  8.24664831e-02
 -1.59858782e-02 -7.79032102e-03 -1.71448123e-02  7.63704553e-02
  1.10217527e-01 -2.29894184e-02  8.37780535e-03  7.16604758e-03
 -5.05505642e-03 -4.04071063e-02  3.40001993e-02  3.00653689e-02
  4.23339978e-02 -2.88223512e-02  3.75781395e-02 -4.33307998e-02
  2.17319671e-02  8.27985723e-03 -1.04310159e-02  7.60996789e-02
  3.55797145e-03  4.72234227e-02  3.57121602e-02  7.10471943e-02
 -2.74737310e-02  2.82491092e-03  1.41288508e-02 -3.47286239e-02
  4.96902678e-04 -3.59864123e-02  5.80329797e-04  1.07831694e-02
 -1.44696620e-03  4.60125

## Setup Pinecone

Primeiramente, cria uma conta lá no [Pinecone](https://app.pinecone.io/) é facinho! Não se esquece de salvar sua API Key em um local seguro, é importante que você não esqueça.

---

Com esse código você pode colocar sua API Key aqui sem se preocupar, pois ela não ficará visível graças aos módulos os e getpass.

In [80]:
os.environ["PINECONE_API_KEY"] = getpass.getpass("Forneça sua API Key do Pinecone: ")

Forneça sua API Key do Pinecone: ··········


In [81]:
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]

Informação trazida da documentação do Pinecone

---

Uma etapa crucial no pinecone é criar um index para armazenar os dados.
O index é criado com um nome, a dimenção do modelo de embeddings que estamos usando e a métrica usada para a busca semântica e por último a cloud a qual você deseja hospedar.

---

Vale lembrar que toda infra do banco de dados (Pinecone) é gerenciada por eles, então só escolha sua cloud e relaxe! xD


In [83]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)

index_name = "similarity-search"

pinecone.create_index(
    name=index_name,
    dimension=model.get_sentence_embedding_dimension(),
    metric='cosine',
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [84]:
index = pinecone.Index(index_name)
print(index)

## Criando os Embeddings e enviando para o Pinecone

Primeiramente criamos um limite de dados na variável "vector_limit", para não sobrecarregar nosso Pinecone (já que estamos utilizando um plano gratuito e limitado). O "batch_size" aqui, serve para processarmos grandes volumes de dados em lotes, otimizando o uso de memória ^^!

---

Pesquisando, descobri que o processamento em lotes é muito utilizado quando lidamos com grandes quantidades de dados. Nesse caso, processamos os dados em lotes menores (no nosso caso de uso, de 200 em 200) até o final da lista.

Pra garantir que o número do lote nunca seja maior que o tamanho da lista de dados, utilizamos a função min(). Ela calcula o índice final de cada iteração de forma segura, evitando ultrapassar o tamanho da lista.

Por exemplo: em uma lista com 5 itens e batch_size = 3, na primeira iteração, o código vai pegar os itens dos índices 0 a 2 (ou seja, 3 itens).

1ª Iteração: Para a lista ["casa", "carro", "coisa", "cuidado", "cheiro"], o lote vai de índice 0 a 2 (ou seja, o lote inclui "casa", "carro" e "coisa").

2ª Iteração: Agora, o código tenta pegar mais 3 itens, mas restam apenas 2 (os itens "cuidado" e "cheiro"). A função min(i + batch_size, len(lista)) garante que o índice final do lote seja ajustado, processando até o índice 4.

Com essa abordagem, conseguimos dividir o processamento de forma eficiente, evitando o risco de tentar acessar um índice fora do limite e, ao mesmo tempo, otimizar o uso de memória e o desempenho.

---

Dando continuidade, precisamos criar os ids para nossos dados. Pra isso usamos o Range para percorrer os lotes e transformamos os números gerados em Strings.

---

Podemos criar metadados associados com o conteúdo que quisermos, nesse caso temos as perguntas associadas a cada id.

---

Por último mas não menos importante, criamos embeddings com nosso modelo escolhido anteriormente, para cada grupo de questões processadas.

---

zipamos tudo, e enviamos para o pinecone.

---

Ufa... Acabei a explicação por aqui.

---

Usamos o TDQM para deixar o processo mais bonitinho, a gente consegue acompanhar visualmente o loop.





In [85]:
batch_size=200
vector_limit=10000

questions = questions[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
    i_end = min(i+batch_size, len(questions))
    ids = [str(x) for x in range(i, i_end)]
    metadatas = [{'text': text} for text in questions[i:i_end]]
    embd = model.encode(questions[i:i_end])
    records = zip(ids, embd, metadatas)
    index.upsert(vectors=records)


  0%|          | 0/50 [00:00<?, ?it/s]

In [86]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 9400}},
 'total_vector_count': 9400}

In [87]:
def run_query(query):
    embedding = model.encode(query).tolist()
    results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
    for result in results['matches']:
        print(f"{round(result['score'], 2)}: {result['metadata']['text']}")


In [88]:
run_query('which city has the highest population in the world?')

0.64: Which is the largest country in the world?
0.6: What percentage of the world's population lives in developed countries?
0.57: Why does America have the most unsafe cities in the world?
0.56: Which is the best city in India?
0.55: Which is the highest mountain in the world?
0.55: About 50% of the world population is concentrated between the latitudes of?
0.54: Which city is the best in India? Why?
0.54: Which country is the world's largest democracy?
0.51: Which is the safest country in the world?
0.51: What do you think are the top 3 countries to live in?


In [89]:
query = 'how do i make a cake?'
run_query(query)

0.62: How do we bake cake in microwave oven?
0.61: How do you make pancakes?
0.54: How do I make whipped cream?
0.51: What are some really simple recipes?
0.51: How do I make a red fondant?
0.51: How can you make duck sauce?
0.48: What does red velvet cake taste like?
0.46: How do I make chili?
0.44: What is your best recipe?
0.44: How do you make a chili dog?
